For citation information, please see the "Source Information" section listed in the associated README file: https://github.com/stephbuon/digital-history/tree/master/hist3368-week6-measuring-change-and-using-groupby

# Week 6: Datetime Applied to Congress

When working with temporal data, we often want to understand the meaning of some variable -- for instance year or decade.  Understanding often implies using that variable to count other variables, for example: how many times do words appear in each decade?  Which words change over time?
    
In this exercise, we will use two strategies to understand wordcount by period: one is the groupby() command, and the other (which you have seen before) is the "for" loop.  The two essentially do the same thing, but they are used in Python in slightly different contexts.

Before we learn about those strategies, however, let's load some data, put the date column into datetime format, and round it to the nearest month, much as we did above.

### Code example:

The code below is the basic code to load debates from the Stanford database of American Congressional debates.  It takes speeches in one dataframe, and "descriptions" of the speeches (who spoke and when) in another dataframe, and then merges them into one database.

Depending on how the Max and Min Threshold are set, you can ingest more speeches or fewer ones.  If you ingest too many speeches at once, you might crash the computer; but if you're working on an HPC machine, you can always restart the session with more memory and try the exercise again.

In [1]:
import datetime
import pandas as pd
import numpy as np
import csv
import glob

The following lines load some data from Congress. Don't worry too much about the commands within this block; we're more interested in the transformations we'll apply to the data after it's loaded.  If you're curious, the lines below download two separate dataframes --  "speeches" and "descriptions" -- and then merge them  so that we now have one database of speeches with the date on which they were spoken.

In [2]:
all_speech_files = glob.glob('/scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_*.txt')
CONGRESS_MIN_THRESHOLD = 90
CONGRESS_MAX_THRESHOLD = 115

speech_files = []

for fn in all_speech_files:
    number = int(fn.rsplit('_', 1)[-1].split('.')[0])
    if CONGRESS_MIN_THRESHOLD <= number <= CONGRESS_MAX_THRESHOLD:
        speech_files.append(fn)

speech_files.sort()
        
def parse_one(fn):
    print(f'Reading {fn}...')
    return pd.read_csv(fn, sep='|', encoding="ISO-8859-1", error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE)

speeches_df = pd.concat((parse_one(fn) for fn in speech_files))
speeches_df.dropna(how='any', inplace=True)

all_description_files = glob.glob('/scratch/group/oit_research_data/stanford_congress/hein-bound/descr_*.txt')
                                  
description_files = []

for fn in all_description_files:
    number = int(fn.rsplit('_', 1)[-1].split('.')[0])
    if CONGRESS_MIN_THRESHOLD <= number <= CONGRESS_MAX_THRESHOLD:
        description_files.append(fn)
        description_files.sort()
        
description_df = pd.concat((parse_one(fn) for fn in description_files))

all_data = pd.merge(speeches_df, description_df, on = 'speech_id')
all_data.fillna(0, inplace=True)


Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_090.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_091.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_092.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_093.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_094.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_095.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_096.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_097.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_098.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_099.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_100.txt...
Reading /s

The data we are looking at is different from the lists of words we worked with previously. This is tabular data -- data with columns, each of which has its own name.  However, all of the commands we've used to work with data before will work.

The **.head()** command is useful for calling up the first few lines of a dataframe so that you can inspect it. 

In [3]:
all_data.head()

,speech_id,speech,chamber,date,number_within_file,speaker,first_name,last_name,state,gender,line_start,line_end,file,char_count,word_count
0,900000001,Those who do not enjoy the privilege of the fl...,S,19670110,1,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,47,49,01101967.txt,86,16
1,900000002,Mr. President. on the basis of an agreement re...,S,19670110,2,Mr. MANSFIELD,Unknown,MANSFIELD,Unknown,M,50,55,01101967.txt,203,35
2,900000003,The Members of the Senate have heard the remar...,S,19670110,3,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,56,62,01101967.txt,234,40
3,900000004,The Chair lays before the Senate the following...,S,19670110,4,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,66,94,01101967.txt,934,151
4,900000005,Secretary of State.,S,19670110,5,Mrs. AGNES BAGGETT,AGNES,BAGGETT,Unknown,F,1047,1048,01101967.txt,19,3


With tabular data, you can call individual columns by name using square brackets.

In [4]:
all_data['speaker']

0                  The VICE PRESIDENT
1                       Mr. MANSFIELD
2                  The VICE PRESIDENT
3                  The VICE PRESIDENT
4                  Mrs. AGNES BAGGETT
                      ...            
5992063                   Ms. GRANGER
5992064    Ms. KILPATRICK of Michigan
5992065                    Mr. HELLER
5992066                   Mr. PAULSEN
5992067          Mr. HALL of New York
Name: speaker, Length: 5992068, dtype: object

You can also use many of the commands we already know.  Here, with .value_counts(), are the ten speakers who spoke the most speeches overall, together with the number of their speeches in our dataset:

In [5]:
all_data['speaker'].value_counts()[:10]

The PRESIDING OFFICER               709041
The SPEAKER pro tempore             239201
The CHAIRMAN                        137788
The SPEAKER                          86866
Mr. ROBERT C. BYRD                   75733
Mr. BYRD                             50374
Mr. DOLE                             48019
The ACTING PRESIDENT pro tempore     46052
Mr. MANSFIELD                        45210
Mr. BAKER                            43007
Name: speaker, dtype: int64

### Select just the most relevant columns

Let's clean up the data a little by deleting columns we won't use in this exercise.   The general command is 

        df.drop('colname', axis)
        
        
Axis is '1' if you're deleting normal columns (not rows).

In [6]:
all_data = all_data.drop(['chamber', 'speech_id', 'number_within_file', 'first_name'], 1)
all_data = all_data.drop(['last_name', 'state', 'gender', 'line_start', 'line_end', 'file', 'char_count'], 1)

In [7]:
all_data.head()

,speech,date,speaker,word_count
0,Those who do not enjoy the privilege of the fl...,19670110,The VICE PRESIDENT,16
1,Mr. President. on the basis of an agreement re...,19670110,Mr. MANSFIELD,35
2,The Members of the Senate have heard the remar...,19670110,The VICE PRESIDENT,40
3,The Chair lays before the Senate the following...,19670110,The VICE PRESIDENT,151
4,Secretary of State.,19670110,Mrs. AGNES BAGGETT,3


### Inspect the data

Now that we have the data in place, let's ask the data: when does it stop or start?

In [8]:
max(all_data['date'])

20101222

The last date is listed above -- year first, then month, then day.

In [9]:
min(all_data['date'])

19670110

Whoops, that's not right.  I bet there are some zeros in the date data for speeches that haven't been assigned a date.  Let's get rid of that data before it messes up our other counts!

Use datetime to render the date column as recognizable dates

In [10]:
all_data['date']=pd.to_datetime(all_data['date'],format='%Y%m%d')
all_data.head()

,speech,date,speaker,word_count
0,Those who do not enjoy the privilege of the fl...,1967-01-10,The VICE PRESIDENT,16
1,Mr. President. on the basis of an agreement re...,1967-01-10,Mr. MANSFIELD,35
2,The Members of the Senate have heard the remar...,1967-01-10,The VICE PRESIDENT,40
3,The Chair lays before the Senate the following...,1967-01-10,The VICE PRESIDENT,151
4,Secretary of State.,1967-01-10,Mrs. AGNES BAGGETT,3


Notice how the results are different after we use pd.to_datetime():

In [11]:
max(all_data['date'])

Timestamp('2010-12-22 00:00:00')

In [12]:
min(all_data['date'])

Timestamp('1967-01-10 00:00:00')

In [13]:
all_data['year'] = pd.to_datetime(all_data['date']).dt.year

all_data.head()

,speech,date,speaker,word_count,year
0,Those who do not enjoy the privilege of the fl...,1967-01-10,The VICE PRESIDENT,16,1967
1,Mr. President. on the basis of an agreement re...,1967-01-10,Mr. MANSFIELD,35,1967
2,The Members of the Senate have heard the remar...,1967-01-10,The VICE PRESIDENT,40,1967
3,The Chair lays before the Senate the following...,1967-01-10,The VICE PRESIDENT,151,1967
4,Secretary of State.,1967-01-10,Mrs. AGNES BAGGETT,3,1967


In [14]:
all_data['month']= pd.to_datetime(all_data['date']).dt.month
all_data.tail()

,speech,date,speaker,word_count,year,month
5992063,Madam Speaker. on rollcall Nos. 662 and 661. I...,2010-12-22,Ms. GRANGER,23,2010,12
5992064,Madam Speaker. as I leave Congress as the peop...,2010-12-22,Ms. KILPATRICK of Michigan,221,2010,12
5992065,Madam Speaker. on rolicall No. 658. I was unav...,2010-12-22,Mr. HELLER,19,2010,12
5992066,Madam Speaker. on rollcall No. 658 my flight w...,2010-12-22,Mr. PAULSEN,23,2010,12
5992067,Madam Speaker. I wold like to submit the follo...,2010-12-22,Mr. HALL of New York,446,2010,12


Notice that the available fields above are the full date time -- which tells us even the minute, when we look closely (by using max() for example), and year.  But I'd like us to be able to count the top words per month. Let's use our "period" command to floor every date to the nearest month, creating a new column called "month_year." 

Look at the data table below to make sure you understand what the new column looks like.

In [15]:
# create a period variable called month_year
all_data['month_year'] = pd.to_datetime(all_data['date']).dt.to_period('M')
all_data.head()

,speech,date,speaker,word_count,year,month,month_year
0,Those who do not enjoy the privilege of the fl...,1967-01-10,The VICE PRESIDENT,16,1967,1,1967-01
1,Mr. President. on the basis of an agreement re...,1967-01-10,Mr. MANSFIELD,35,1967,1,1967-01
2,The Members of the Senate have heard the remar...,1967-01-10,The VICE PRESIDENT,40,1967,1,1967-01
3,The Chair lays before the Senate the following...,1967-01-10,The VICE PRESIDENT,151,1967,1,1967-01
4,Secretary of State.,1967-01-10,Mrs. AGNES BAGGETT,3,1967,1,1967-01


Next, we're going to convert the period column that we just created -- month_year -- back to date format (it's now in datetime's special "month-year" format). This is basically an issue that we're dealing with now so that our graphics work later.  Matplotlib knows what to do with full dates, but it doesn't know what to do with "month-year" format. You don't need to understand the following line of code, but you need to know that sometimes converting periods back to datetime format will resolve graphics issues.

In [16]:
# convert month-year back to date format for easier plotting later (this normalizes the field to a kind of date that matplotlib likes)
all_data['month_year'] =all_data.month_year.values.astype('datetime64[M]')
all_data.head()

,speech,date,speaker,word_count,year,month,month_year
0,Those who do not enjoy the privilege of the fl...,1967-01-10,The VICE PRESIDENT,16,1967,1,1967-01-01
1,Mr. President. on the basis of an agreement re...,1967-01-10,Mr. MANSFIELD,35,1967,1,1967-01-01
2,The Members of the Senate have heard the remar...,1967-01-10,The VICE PRESIDENT,40,1967,1,1967-01-01
3,The Chair lays before the Senate the following...,1967-01-10,The VICE PRESIDENT,151,1967,1,1967-01-01
4,Secretary of State.,1967-01-10,Mrs. AGNES BAGGETT,3,1967,1,1967-01-01


In [21]:
# list all unique month_years
set(all_data['month_year'])[:10]

TypeError: 'set' object is not subscriptable

Notice that, even though we have a full datetime stamp for that column, there's only one date listed per month and per year.  This means that March 26, 2010 has been rounded to 2010-03-01, and will be grouped with March 22 and March 15 of 2010.  

Now that we have a standardize month_year variable, we can ask the computer to group all words that were spoken in that month together and measure how many words, total, were spoken in that month.  

To count all the words spoken in every month_year, we'll need a new tool from pandas: groupby(). 

#### Let's save a copy for later on your account to make reading data again simple.

Data analysts frequently save local backup copies of data so that they don't have to reload the whole thing again. 

First, navigate to your home folder.

In [19]:
cd ~/digital-history

/users/jguldi/digital-history


We can use the 

    .to_csv()

function to store the pandas dataframe as a csv, or comma separated values sheet.  


Using the left-hand sidebar, navigate to the digital-history folder and notice that you've saved a copy of your data there.

You can download the resulting file and open it to inspect it on your home computer.  A CSV can be read by programs like Excel.

In [20]:
all_data.to_csv("congress1967-2010.csv")

When we need to read it again, we can call it with this line:
    
    pd.read_csv("congress1967-2010.csv")